# VectorStore-backed memory

The support for Cassandra vector store, available in LangChain, enables another interesting use case, namely a chat memory buffer that injects the most relevant past exchanges into the prompt, instead of the most recent (as most other memories do). This enables retrieval of related context _arbitrarily far back in the chat history_. All you need is to instantiate a `Cassandra` vector store and wrap it in a `VectorStoreRetrieverMemory` type of memory, provided by LangChain.

## Colab-specific setup

Make sure you have a database available. This is a Colab notebook, so here it is assumed you will use
an Astra DB cloud instance. For usage with an Apache Cassandra® cluster, switch to a local setup
as instructed [on cassio.org](https://cassio.org/more_info/#use-a-local-vector-capable-cassandra).

Get ready to supply the connection parameters: Database ID and Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details.
Remember you need a Token with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Likewise, ensure you have the necessary secret(s) for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/memory-vectorstore/)._


In [ ]:
# install required dependencies
! pip install -q --progress-bar off \
    "langchain>=0.0.348" \
    "cassio>=0.1.3" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2" 
exit()

⚠️ **Do not mind a "Your session crashed..." message you may see.**

It was us, making sure your kernel restarts with all the correct dependency versions. _You can now proceed with the notebook._

### DB connection parameters
Example values (check your Astra dashboard):
- Database ID example: `01234567-89ab-cdef-0123-456789abcdef`
- Token example: `AstraCS:6gBhNmsk135....` (ensure it has a role of at least "Database Administrator")
- _Keyspace. Optional, string if provided. Example:_ `default_keyspace`

In [ ]:
import os
from getpass import getpass

# Input your Astra DB connection parameters and secrets:

os.environ["ASTRA_DB_ID"] = input("ASTRA_DB_ID = ")

os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

_keyspace = input("(Optional) ASTRA_DB_KEYSPACE = ")
if _keyspace:
    os.environ["ASTRA_DB_KEYSPACE"] = _keyspace

### LLM Provider

In the cell below you can choose between **GCP Vertex AI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [ ]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'


In [ ]:
from getpass import getpass
if llmProvider == 'OpenAI':
    apiSecret = getpass(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
elif llmProvider == 'Azure_OpenAI':
    # a few parameters must be input
    apiSecret = input(f'Your API Key for LLM provider "{llmProvider}": ')
    os.environ['AZURE_OPENAI_API_KEY'] = apiSecret
    apiBase = input('The "Base URL" for your models (e.g. "https://YOUR-RESOURCE-NAME.openai.azure.com"): ')
    os.environ['AZURE_OPENAI_API_BASE'] = apiBase
    apiLLMDepl = input('The name of your LLM Deployment: ')
    os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'] = apiLLMDepl
    apiLLMModel = input('The name of your LLM Model (e.g. "gpt-4"): ')
    os.environ['AZURE_OPENAI_LLM_MODEL'] = apiLLMModel
    apiEmbDepl = input('The name for your Embeddings Deployment: ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'] = apiEmbDepl
    apiEmbModel = input('The name of your Embedding Model (e.g. "text-embedding-ada-002"): ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'] = apiEmbModel

    # The following is probably not going to change for some time...
    os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'
else:
    raise ValueError('Unknown/unsupported LLM Provider')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# VectorStore-backed memory

The support for Cassandra vector store, available in LangChain, enables another interesting use case, namely a chat memory buffer that injects the most relevant past exchanges into the prompt, instead of the most recent (as most other memories do). This enables retrieval of related context _arbitrarily far back in the chat history_.

All you need is to instantiate a `Cassandra` vector store and wrap it in a `VectorStoreRetrieverMemory` type of memory, provided by LangChain.

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [2]:
from langchain.vectorstores.cassandra import Cassandra

A database connection is needed. _(If on a Colab, the only supported option is the cloud service Astra DB.)_

In [3]:
# Getting ready to initialize the DB connection globally ...
import os

import cassio

Select your choice of database by editing this cell, if needed:

In [4]:
database_mode = "astra_db"  # only "astra_db" supported on Colab

In [5]:
if database_mode == "astra_db":
    cassio.init(
        database_id=os.environ["ASTRA_DB_ID"],
        token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
        keyspace=os.environ.get("ASTRA_DB_KEYSPACE"),  # this is optional
    )

In [6]:
if database_mode == "cassandra":
    # Cassandra not supported on Colab - please define your own getCassandraCQLSession/getCassandraCQLKeyspace
    cassio.init(
        session=getCassandraCQLSession(),
        keyspace=getCassandraCQLKeyspace(),
    )

Both an LLM and an embedding function are required.

Below is the logic to instantiate the LLM and embeddings of choice. We chose to leave it in the notebooks for clarity.

In [7]:
import os
# creation of the LLM resources


if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    myEmbedding = VertexAIEmbeddings()
    print('LLM+embeddings from Vertex AI')
elif llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from langchain.llms import OpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = OpenAI(temperature=0)
    myEmbedding = OpenAIEmbeddings()
    print('LLM+embeddings from OpenAI')
elif llmProvider == 'Azure_OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'azure'
    os.environ['OPENAI_API_VERSION'] = os.environ['AZURE_OPENAI_API_VERSION']
    os.environ['OPENAI_API_BASE'] = os.environ['AZURE_OPENAI_API_BASE']
    os.environ['OPENAI_API_KEY'] = os.environ['AZURE_OPENAI_API_KEY']
    from langchain.llms import AzureOpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = AzureOpenAI(temperature=0, model_name=os.environ['AZURE_OPENAI_LLM_MODEL'],
                      engine=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'])
    myEmbedding = OpenAIEmbeddings(model=os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'],
                                   deployment=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'])
    print('LLM+embeddings from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from OpenAI


## Create the store

In [8]:
table_name = 'vstore_memory_' + llmProvider
cassVStore = Cassandra(
    session=None,
    keyspace=None,
    table_name=table_name,
    embedding=myEmbedding,
)

# just in case this demo runs multiple times
cassVStore.clear()

## Create the retriever and the memory

From the vector store a "retriever" is spawned. You'll keep the number of items to fetch intentionally very small for demonstration purposes.

Next, the retriever is wrapped in a `VectorStoreRetrieverMemory`:

In [9]:
retriever = cassVStore.as_retriever(search_kwargs={'k': 3})
semanticMemory = VectorStoreRetrieverMemory(retriever=retriever)

Create a fake "past conversation". Note how the topic of the discussion wanders to fixing one's PC in the last few exchanges:

In [10]:
pastExchanges = [
    (
        {"input": "Hello, what is the biggest mammal?"},
        {"output": "The blue whale."},
    ),
    (
        {"input": "... I cannot swim. Actually I hate swimming!"},
        {"output": "I see."},
    ),
    (
        {"input": "I like mountains and beech forests."},
        {"output": "That's good to know."},
    ),
    (
        {"input": "Yes, too much water makes me uneasy."},
        {"output": "Ah, how come?."},
    ),
    (
        {"input": "I guess I am just not a seaside person"},
        {"output": "I see. How may I help you?"},
    ),
    (
        {"input": "I need help installing this driver"},
        {"output": "First download the right version for your operating system."},
    ),
    (
        {"input": "Good grief ... my keyboard does not work anymore!"},
        {"output": "Try plugging it in your PC first."},
    ),
]

Insert these exchanges into the memory:

In [11]:
for exI, exO in pastExchanges:
    semanticMemory.save_context(exI, exO)

Given a conversation input, the `load_memory_variables` performs a semantic search and comes up with relevant items from the memory, regardless of their order:

In [12]:
QUESTION = "Can you suggest me a sport to try?"

In [13]:
print(semanticMemory.load_memory_variables({"prompt": QUESTION})["history"])

input: ... I cannot swim. Actually I hate swimming!
output: I see.
input: I guess I am just not a seaside person
output: I see. How may I help you?
input: I like mountains and beech forests.
output: That's good to know.


## Usage in a conversation chain

This semantic memory element can be used within a full conversation chain.

In the following you'll create a custom prompt and a `ConversationChain` out of it, attaching the latter to the vector-store-powered memory seen above:

In [14]:
semanticMemoryTemplateString = """The following is a between a human and a helpful AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

The AI can use information from parts of the previous conversation (only if they are relevant):
{history}

Current conversation:
Human: {input}
AI:"""

memoryPrompt = PromptTemplate(
    input_variables=["history", "input"],
    template=semanticMemoryTemplateString
)

conversationWithVectorRetrieval = ConversationChain(
    llm=llm, 
    prompt=memoryPrompt,
    memory=semanticMemory,
    verbose=True
)

Run the chain with the sports question:

In [15]:
conversationWithVectorRetrieval.predict(input=QUESTION)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a between a human and a helpful AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

The AI can use information from parts of the previous conversation (only if they are relevant):
input: ... I cannot swim. Actually I hate swimming!
output: I see.
input: I guess I am just not a seaside person
output: I see. How may I help you?
input: I like mountains and beech forests.
output: That's good to know.

Current conversation:
Human: Can you suggest me a sport to try?
AI:

> Finished chain.


' Sure, I can suggest some sports for you to try. Depending on your preferences, you could try hiking, running, biking, or even rock climbing. Do any of these sound interesting to you?'

Notice how new exchanges are automatically added to the memory:

In [16]:
conversationWithVectorRetrieval.predict(input="Would I like a swim in a mountain lake?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a between a human and a helpful AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

The AI can use information from parts of the previous conversation (only if they are relevant):
input: I like mountains and beech forests.
output: That's good to know.
input: Can you suggest me a sport to try?
response:  Sure, I can suggest some sports for you to try. Depending on your preferences, you could try hiking, running, biking, or even rock climbing. Do any of these sound interesting to you?
input: ... I cannot swim. Actually I hate swimming!
output: I see.

Current conversation:
Human: Would I like a swim in a mountain lake?
AI:

> Finished chain.


" That depends on your preferences. If you don't like swimming, then a swim in a mountain lake may not be the best activity for you. However, if you enjoy the outdoors and the beauty of nature, then a swim in a mountain lake could be a great experience."

... so that now the most relevant items for the same question are changed:

In [17]:
semanticMemory.retriever.get_relevant_documents(QUESTION)

[Document(page_content='input: Can you suggest me a sport to try?\nresponse:  Sure, I can suggest some sports for you to try. Depending on your preferences, you could try hiking, running, biking, or even rock climbing. Do any of these sound interesting to you?', metadata={}),
 Document(page_content="input: Would I like a swim in a mountain lake?\nresponse:  That depends on your preferences. If you don't like swimming, then a swim in a mountain lake may not be the best activity for you. However, if you enjoy the outdoors and the beauty of nature, then a swim in a mountain lake could be a great experience.", metadata={}),
 Document(page_content='input: ... I cannot swim. Actually I hate swimming!\noutput: I see.', metadata={})]

## A counterexample

What would happen with a simpler memory element, which simply retrieves a certain number of most recent interactions?

Create and populate an instance of LangChain's `ConversationTokenBufferMemory`, limiting it to a maximum token length of 80 (roughly equivalent to the 3 fragments set for the `semanticMemory` object):

In [18]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ChatMessageHistory

In [19]:
baseHistory = ChatMessageHistory()

recencyBufferMemory = ConversationTokenBufferMemory(
    chat_memory=baseHistory,
    max_token_limit=80,
    llm=llm,
)

In [20]:
for exI, exO in pastExchanges:
    recencyBufferMemory.save_context(exI, exO)

Time to ask the same sports question. This is what will get injected into the prompt this time:

In [21]:
print(recencyBufferMemory.load_memory_variables({"prompt": QUESTION})["history"])

AI: Ah, how come?.
Human: I guess I am just not a seaside person
AI: I see. How may I help you?
Human: I need help installing this driver
AI: First download the right version for your operating system.
Human: Good grief ... my keyboard does not work anymore!
AI: Try plugging it in your PC first.


... and this is the (rather generic) answer you'd get:

In [22]:
conversationWithRecencyRetrieval = ConversationChain(
    llm=llm, 
    prompt=memoryPrompt,
    memory=recencyBufferMemory,
    verbose=True
)

conversationWithRecencyRetrieval.predict(input=QUESTION)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a between a human and a helpful AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

The AI can use information from parts of the previous conversation (only if they are relevant):
AI: Ah, how come?.
Human: I guess I am just not a seaside person
AI: I see. How may I help you?
Human: I need help installing this driver
AI: First download the right version for your operating system.
Human: Good grief ... my keyboard does not work anymore!
AI: Try plugging it in your PC first.

Current conversation:
Human: Can you suggest me a sport to try?
AI:

> Finished chain.


' Sure! What kind of sport are you interested in?'

## What now?

This demo is hosted [here](https://cassio.org/frameworks/langchain/memory-vectorstore/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/langchain/about/)**.